In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [5]:
# Set the tournament name with the year and city of the regional

tournament = 'pokemon-toronto-2025'

homepage = 'https://rk9.gg/event/' + tournament

In [6]:
def get_rk9_urls(homepage):
    soup = BeautifulSoup(requests.get(homepage).text)

    tcg = soup.find("div", class_ = 'card h-100 mt-3 p-2 shadow bg-blue-050') # Locate tcg box

    # Find url extensions for roster and pairings
    roster_code = tcg.find('a', {'href': re.compile('/roster*')})['href']  
    pairings_code = tcg.find('a', {'href': re.compile('/pairings*')})['href']

    roster_url = 'https://rk9.gg' + roster_code
    pairings_url = 'https://rk9.gg' + pairings_code
    
    return [roster_url,pairings_url]

In [7]:
def get_roster(homepage, csv = False, filename = tournament + 'roster'):
    
    url = get_rk9_urls(homepage)[0] #set player roster url
    
    soup = BeautifulSoup(requests.get(url).text) #load in the soup

    table = soup.find('table') #roster page only has one table
    
    headers = table.find_all('th') #Find the column headers
    headers = [heading.string for heading in headers]
    
    body = table.find('tbody') # isolate the body of the table
    
    rows = body.find_all('tr') # get rows
    
    all_roster_data = []
    for row in rows:  # This loop isolates the text in each cell

        row_data = row.find_all('td')
        individual_row_data = [data.text.strip() for data in row_data]

        dlist = row_data[-2]
        dlist_url = dlist.find('a')['href']  # This is grabbing the decklist url, otherwise you just get "view"

        individual_row_data[-2] = dlist_url

        all_roster_data.append(individual_row_data)
        
        df = pd.DataFrame(all_roster_data, columns = headers)
        
    if not csv:
        return df #returns the desired table
    if csv:
        return df.to_csv(filename)

In [8]:
get_roster(homepage)

,Player ID,First name,Last name,Country,Division,Deck List,Standing
0,3....5,Jen,Snazyk,CA,Masters,/decklist/public/TO01mePm6kO6bDc2IN6U/02ktBAK5...,2088
1,4....5,Matthew,C.,CA,Junior,/decklist/public/TO01mePm6kO6bDc2IN6U/02mP7ZnG...,104
2,4....7,Xavier,Ménard-Beaudoin,CA,Masters,/decklist/public/TO01mePm6kO6bDc2IN6U/03QyZ6DR...,1084
3,4....6,Eizaku,Asai,US,Masters,/decklist/public/TO01mePm6kO6bDc2IN6U/03jChi5D...,1972
4,1....5,William,Illig,US,Masters,/decklist/public/TO01mePm6kO6bDc2IN6U/03lydN7Y...,1718
...,...,...,...,...,...,...,...
2521,4....9,Kevin,Fuentes,US,Masters,/decklist/public/TO01mePm6kO6bDc2IN6U/zuxR4vhi...,188
2522,5....9,Sackett,Brehmer,US,Masters,/decklist/public/TO01mePm6kO6bDc2IN6U/zwgdtpvK...,526
2523,2....8,Vincenzo,Porco,CA,Masters,/decklist/public/TO01mePm6kO6bDc2IN6U/zxN4Z8F4...,1374
2524,4....1,Andrew,Pham,CA,Masters,/decklist/public/TO01mePm6kO6bDc2IN6U/zy9WG6hL...,1885


In [ ]:
dlist_url = 
dlist_soup = BeautifulSoup(requests.get("https://rk9.gg/" + dlist_url).text)